# Data Collection

This assumes that you have registered an account with Kaggle, and have created a json key to interact with Kaggle's API.  

In [135]:
import os
import shutil
import pandas as pd
import numpy as np

In [111]:
os.chdir(os.path.expanduser('~'))
os.chdir('Desktop/skin')

current_dir = os.getcwd()
current_dir

'/home/pxp/Desktop/skin'

In [112]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

In [113]:
meta_data = pd.read_csv("inputs/cancer-dataset/HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


# Data Preparation

## Data Cleaning
Check and remove non-image files

In [114]:
my_data_dir='inputs/cancer-dataset/photos'

## Split train validation test set

In [115]:
meta_data = pd.read_csv("inputs/cancer-dataset/HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [116]:
image_dir_path = 'inputs/cancer-dataset/photos' 
train_dir = 'inputs/cancer-dataset/train'
test_dir = 'inputs/cancer-dataset/test'
val_dir = 'inputs/cancer-dataset/validate'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

cancer_types = meta_data["dx"].unique()
for cancer_type in cancer_types:
    os.makedirs(os.path.join(train_dir, cancer_type), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cancer_type), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cancer_type), exist_ok=True)


In [117]:
meta_data = meta_data.sample(frac=1).reset_index(drop=True)

test_count = int(meta_data.shape[0]*.7)
val_count = int(meta_data.shape[0]*.2)

for i in range(meta_data.shape[0]):
    image_path = os.path.join(image_dir_path, meta_data["image_id"][i]+".jpg")
    if i < test_count:
        shutil.copy(image_path, os.path.join(train_dir, meta_data["dx"][i]))
    elif i < test_count + val_count:
        shutil.copy(image_path, os.path.join(test_dir, meta_data["dx"][i]))
    else:
        shutil.copy(image_path, os.path.join(val_dir, meta_data["dx"][i]))

In [148]:
paths = [train_dir, test_dir, val_dir]
names = ['TRAIN', 'TEST', 'VALIDATE']
nums = pd.DataFrame(columns=names, index=cancer_types)

for name in names: 
    for cancer_type in cancer_types:
        nums.loc[cancer_type, name] = len(os.listdir(os.path.join(paths[names.index(name)], cancer_type)))

nums.loc['TOTAL'] = nums.sum()
nums['TOTAL'] = nums.sum(axis=1)

print('Files found:')
nums 

Files found:


,TRAIN,TEST,VALIDATE,TOTAL
bkl,759,242,98,1099
nv,4720,1326,659,6705
df,82,20,13,115
mel,763,206,144,1113
vasc,104,23,15,142
bcc,360,105,49,514
akiec,222,81,24,327
TOTAL,7010,2003,1002,10015
